In [3]:
import torch
import torch
from sentence_transformers import SentenceTransformer, util

In [2]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f5dad5943ab05e00c212f84d9ace501ee85b7542790c160bbe669987344d32ff
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
import pandas as pd
def get_top_n_relevant_passages(query, passages, n=3):
    model_name = 'msmarco-distilbert-base-tas-b'
    model = SentenceTransformer(model_name)

    # Encode query and passages
    query_embedding = model.encode([query], convert_to_tensor=True)
    passage_embeddings = model.encode(passages, convert_to_tensor=True)

    # Compute cosine similarity between query and passages
    cosine_scores = util.pytorch_cos_sim(query_embedding, passage_embeddings)[0].cpu().numpy()

    # Get the indices of the top N cosine similarity scores
    top_n_indices = np.argsort(cosine_scores)[-n:][::-1]

    # Get the top N relevant passages
    top_n_relevant_passages = [passages[i] for i in top_n_indices]

    return top_n_relevant_passages


In [6]:
df = pd.read_excel('orignal_dataset_1.xlsx')
df = df[['claim', 'evidence']]
text = df['claim'].iloc[0:500]
df.loc[:, 'evidence'] = df['evidence'].apply(str)
sentences = list(df['evidence'])


In [7]:
import numpy as np


In [8]:
lst1 = []
lst2 = []
ground_truth = []

for claim in text:
    print("Claim:", claim)
    top_3_relevant_passages = get_top_n_relevant_passages(claim, sentences, n=3)
    lst1.append(claim)
    lst2.append(top_3_relevant_passages)

    # Assuming that your ground truth data is a list of passages for each claim
    # Replace the line below with the actual ground truth data
    ground_truth.append("ground_truth_passage")  # Single ground truth passage


Claim: Asif Zardari has announced support for Imran Khan, saying Shehbaz Sharif cannot lead Pakistan


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Claim: A video shows a Punjab Police officer “misbehaving” and “insulting” an older woman amid a recent crackdown by authorities following protests across Pakistan against the 9 May arrest of Imran Khan.
Claim: Video shows Akshay Kumar condemning Imran Khan's arrest
Claim: Former prime minister and Pakistan Tehreek-e-Insaf (PTI) chief Imran Khan has died.
Claim: Agreement signed between detained ex-Pak PM Imran Khan, Pakistan government and US ambassador with bizarre “no-rape” conditions
Claim: sri Lanka to gift two female elephants to Pakistan !!
Claim: US President Joe Biden has allocated $500,000 to “transgender Pakistani youth English lessons”.
Claim: New Zealand cricket commentator Simon Doull says living in Pakistan is like living in jail
Claim: Astrologists and geologists predict that a massive earthquake similar to the quake that hit Turkey on Monday will strike India and Pakistan in 15 days.
Claim: Gwadar has a cricket stadium
Claim: Christian man, Mohammad Issa lynched in Pak

In [9]:
lst2

[['Asif Zardari has not announced support for Khan, nor has he said Sharif cannot lead Pakistan. the press conference shown in the video is actually from 2019. Another clip with the same caption but a different press conference is actually from 2022.On 24 May 2023, Facebook page ‘Entertainment Videos’ posted a live video (archive) showing the former president and the co-chairperson of the Pakistan People’s Party (PPP), Asif Ali Zardari, addressing a press conference.The video — which had over 520,000 views, 600 comments, and 500 shares as of writing time.',
  'Hrithik Roshan did not voice support for Imran Khan.A clip from an old interaction of Roshan with his fans has been doctored. The actor has not actually made any comments regarding Imran Khan or voiced his support for the politician.On 11 May 2023, Facebook user Sohag Ali Zaman falsely shared a video with the claim that actor Hrithik Roshan has requested his fans to support Pakistan Tehreek-e-Insaf chief Imran Khan following his 

In [10]:
ground = df['evidence'].iloc[0:100]

In [11]:
rr_list = []

for true_passage, predicted_passages in zip(ground, lst2):
    # Find the position of the true_passage in predicted_passages
    try:
        position = predicted_passages.index(true_passage) + 1
        rr = 1 / position
    except ValueError:
        # If true_passage is not in predicted_passages, set rr to 0
        rr = 0

    rr_list.append(rr)

# Calculate Mean Reciprocal Rank
mrr = sum(rr_list) / len(rr_list)

print("Mean Reciprocal Rank:", mrr)

Mean Reciprocal Rank: 0.94
